# 장애인콜택시 대기시간 예측
## 단계1.데이터 전처리
## 단계2.데이터 분석
## 단계3.모델링 및 예측

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

# 더 필요한 라이브러리를 아래에 추가합니다.



### (3) 데이터 불러오기
* 주어진 데이터셋
    * 장애인 콜택시 운행 정보 : open_data.csv
    * 날씨 데이터 : weather.csv

#### 1) 데이터로딩

In [6]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi=pd.read_csv('open_data.csv')
weather=pd.read_csv('weather.csv')

#### 2) 기본 정보 조회

In [7]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 전체 데이터의 행,열 개수 확인
print(taxi.shape)
print(weather.shape)

(2922, 7)
(4018, 7)


In [8]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 전체 데이터의 모든 변수 확인
taxi.head()

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


In [9]:
weather.head()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2012-01-01,0.4,-6.6,0.0,77.0,45.0,4.90
1,2012-01-02,-1.2,-8.3,0.0,80.0,48.0,6.16
2,2012-01-03,-0.4,-6.6,0.4,86.0,45.0,4.46
3,2012-01-04,-4.6,-9.5,0.0,66.0,38.0,8.05
4,2012-01-05,-1.4,-9.6,0.0,71.0,28.0,9.14


In [10]:
print(taxi.info())
print(weather.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일     2922 non-null   object 
 1   차량운행    2922 non-null   int64  
 2   접수건     2922 non-null   int64  
 3   탑승건     2922 non-null   int64  
 4   평균대기시간  2922 non-null   float64
 5   평균요금    2922 non-null   int64  
 6   평균승차거리  2922 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 159.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             4018 non-null   object 
 1   temp_max         4018 non-null   float64
 2   temp_min         4018 non-null   float64
 3   rain(mm)         4018 non-null   float64
 4   humidity_max(%)  4018 non-null   float64
 5   humidity_min(%)  4018 non-null   float64
 6   sunshine(MJ/m

#### 3) 칼럼이름을 영어로 변경
* 꼭 필요한 작업은 아니지만, 데이터를 편리하게 다루고 차트에서 불필요한 경고메시지를 띄우지 않게 하기 위해 영문으로 변경하기를 권장합니다.


In [11]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



## 2.데이터 기본 탐색

* **세부요구사항**
    * 날짜 요소에 따라 각 정보의 패턴을 조회 합니다.
        * 일별, 요일별, 주차별, 월별, 연도별
        * 접수건, 탑승건, 거리, 요금, 대기시간 등
    * 제시된 범위 외에 가능하다면 추가 탐색을 시도합니다.

### (1) 주기별 분석을 위해서 날짜 변수 추가하기
* data를 복사합니다.
* 복사한 df에 요일, 주차, 월, 연도 등을 추가합니다.

In [12]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi['기준일']=pd.to_datetime(taxi['기준일'])
weather['Date']=pd.to_datetime(weather['Date'])

In [13]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi=taxi.copy()
df_weather=weather.copy()

In [14]:
df_taxi['year']=df_taxi['기준일'].dt.year
df_taxi['month']=df_taxi['기준일'].dt.month
df_taxi['day']=df_taxi['기준일'].dt.day
df_taxi['weekday']=df_taxi['기준일'].dt.day_name()

In [15]:
df_taxi.tail()

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,year,month,day,weekday
2917,2022-12-27,669,5635,4654,44.4,2198,8178,2022,12,27,Tuesday
2918,2022-12-28,607,5654,4648,44.8,2161,7882,2022,12,28,Wednesday
2919,2022-12-29,581,5250,4247,52.5,2229,8433,2022,12,29,Thursday
2920,2022-12-30,600,5293,4200,38.3,2183,8155,2022,12,30,Friday
2921,2022-12-31,263,2167,1806,33.7,2318,9435,2022,12,31,Saturday


### (2) 일별

* 차량 운행수

In [16]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('day',as_index=True)[['차량운행']].describe()

차량운행                                                              
    count        mean         std    min     25%    50%     75%     max
day                                                                    
1    96.0  388.375000  144.066596  178.0  245.75  414.5  505.75   700.0
2    96.0  419.520833  132.769288  161.0  284.75  444.5  511.00   690.0
3    96.0  408.614583  129.551402  164.0  275.75  444.5  509.25   631.0
4    96.0  418.666667  134.711361  186.0  273.50  451.0  527.00   650.0
5    96.0  408.968750  134.806407  171.0  268.50  441.5  521.00   655.0
6    96.0  409.031250  130.300822  190.0  276.75  444.5  521.25   679.0
7    96.0  427.583333  133.118204  167.0  290.75  458.5  522.00   692.0
8    96.0  428.739583  134.107203  181.0  301.50  461.5  531.75   697.0
9    96.0  409.052083  132.819495  167.0  270.75  442.5  513.50   649.0
10   96.0  416.083333  134.381129  171.0  274.50  454.0  516.00   628.0
11   96.0  421.281250  132.201099  179.0  282.75  453.0  525.00   616.0
12   96.0  415.989583  128.281803  194.0  297.25  449.5  514.50   659.0
13   96.0  417.125000  128.383000  180.0  296.00  451.5  510.00   676.0
14   96.0  425.156250  131.610776  189.0  279.50  458.5  530.00   677.0
15   96.0  412.333333  133.616865  190.0  270.75  447.0  523.25   670.0
16   96.0  420.958333  132.872073  164.0  267.75  457.5  519.00   646.0
17   96.0  422.052083  133.574058  184.0  280.75  459.0  527.50   638.0
18   96.0  422.697917  132.900289  175.0  280.00  458.5  525.25   661.0
19   96.0  421.708333  128.183989  176.0  296.00  464.5  519.00   663.0
20   96.0  418.677083  128.937234  179.0  281.00  450.5  515.75   666.0
21   96.0  426.281250  130.964013  194.0  291.75  468.0  523.25   654.0
22   96.0  428.312500  129.543434  188.0  311.25  460.5  525.50   665.0
23   96.0  440.260417  164.204958  171.0  295.25  465.5  533.25  1413.0
24   96.0  416.031250  136.785768  157.0  275.00  458.5  518.25   667.0
25   96.0  405.729167  138.975233  176.0  247.00  445.5  522.00   653.0
26   96.0  420.239583  128.159583  193.0  273.75  456.0  512.25   660.0
27   96.0  419.541667  132.988002  163.0  273.75  457.0  522.75   669.0
28   96.0  423.416667  131.440254  187.0  277.75  454.5  526.50   670.0
29   90.0  430.522222  136.754535  132.0  300.00  467.0  525.00   814.0
30   88.0  424.147727  136.437981  182.0  276.25  462.0  522.50   685.0
31   56.0  410.392857  136.966974  139.0  290.75  451.5  506.75   682.0

* 접수건, 탑승건

In [17]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('day',as_index=True)[['접수건','탑승건']].describe()

접수건                                                                      \
    count         mean          std     min      25%     50%      75%     max   
day                                                                             
1    96.0  3462.656250  1648.545948   527.0  1836.50  3926.5  5048.25  6051.0   
2    96.0  3925.177083  1498.422026  1114.0  2156.75  4713.0  5100.00  5990.0   
3    96.0  3725.281250  1502.094718   591.0  2060.50  4519.0  4975.00  5715.0   
4    96.0  3886.364583  1472.694953   967.0  2147.50  4543.0  5081.75  5948.0   
5    96.0  3755.000000  1591.985420   623.0  1978.50  4464.0  5084.25  5916.0   
6    96.0  3740.760417  1590.899532   655.0  2003.75  4444.0  5124.25  6092.0   
7    96.0  4030.062500  1498.985410   977.0  2165.00  4811.5  5131.50  5985.0   
8    96.0  4058.052083  1467.594382   596.0  2358.00  4876.5  5122.50  5929.0   
9    96.0  3770.041667  1516.003638  1076.0  2109.00  4663.5  5112.00  5892.0   
10   96.0  3910.520833  1488.604108   624.0  2190.75  4748.0  5076.75  5735.0   
11   96.0  3974.625000  1486.303031  1055.0  2279.00  4822.5  5123.25  5940.0   
12   96.0  3941.635417  1534.408634   748.0  2152.75  4721.0  5123.25  6164.0   
13   96.0  3969.510417  1554.573599   641.0  2250.75  4753.0  5100.00  6133.0   
14   96.0  4019.187500  1502.038743   948.0  2196.25  4822.0  5149.75  5966.0   
15   96.0  3839.427083  1539.956922   590.0  2060.50  4697.5  5102.75  6128.0   
16   96.0  3952.166667  1452.992583  1149.0  2173.00  4706.5  5086.75  5864.0   
17   96.0  3948.937500  1489.228107   698.0  2193.75  4707.0  5107.00  5939.0   
18   96.0  3992.968750  1460.873433   859.0  2262.25  4682.0  5119.75  5997.0   
19   96.0  3976.906250  1520.346478   816.0  2286.25  4766.0  5123.00  6071.0   
20   96.0  4081.114583  1571.507368   743.0  2302.25  4870.5  5177.50  6075.0   
21   96.0  4080.166667  1484.030931  1128.0  2343.75  4862.0  5175.25  6072.0   
22   96.0  4064.052083  1487.780518   626.0  2327.25  4858.0  5161.25  5932.0   
23   96.0  4081.354167  1440.606517   738.0  2302.75  4756.0  5132.50  5913.0   
24   96.0  3891.437500  1462.307562   900.0  2207.25  4613.0  5055.75  5973.0   
25   96.0  3792.614583  1533.567436   929.0  2061.50  4484.0  5048.50  5898.0   
26   96.0  3986.062500  1503.268613  1027.0  2236.25  4761.5  5126.75  5986.0   
27   96.0  3989.750000  1541.460449   609.0  2168.25  4787.0  5133.00  6100.0   
28   96.0  4029.010417  1503.118558  1171.0  2186.25  4758.0  5132.50  6182.0   
29   90.0  4029.000000  1483.460856   618.0  2288.00  4742.0  5123.50  6093.0   
30   88.0  3932.238636  1498.755867   619.0  2214.00  4606.5  5101.50  6134.0   
31   56.0  3821.321429  1507.031738   958.0  2070.25  4622.5  4972.00  5836.0   

      탑승건                                                                      
    count         mean          std     min      25%     50%      75%     max  
day                                                                            
1    96.0  2875.177083  1383.488603   462.0  1602.25  3146.0  4141.75  5129.0  
2    96.0  3277.520833  1241.608608  1048.0  1807.75  3918.0  4222.50  5025.0  
3    96.0  3129.187500  1247.693538   551.0  1734.00  3806.0  4111.75  4742.0  
4    96.0  3269.989583  1216.042074   908.0  1772.50  3863.5  4189.00  5038.0  
5    96.0  3153.093750  1302.277051   591.0  1670.25  3916.5  4129.50  4962.0  
6    96.0  3148.364583  1320.859083   618.0  1670.25  3878.0  4228.50  5083.0  
7    96.0  3377.666667  1239.775554   887.0  1774.25  3973.5  4332.25  4990.0  
8    96.0  3399.833333  1215.058497   550.0  1859.50  4010.5  4249.75  5000.0  
9    96.0  3180.666667  1281.586530   957.0  1719.25  3966.5  4246.75  4939.0  
10   96.0  3277.781250  1231.381588   586.0  1780.75  3976.0  4247.75  4817.0  
11   96.0  3327.687500  1221.741977   990.0  1814.00  3959.5  4268.50  4988.0  
12   96.0  3312.750000  1259.593945   695.0  1836.50  3991.0  4270.50  5098.0  
13   96.0  3311.656250  1269.611162   611.0  1819.7

* 대기시간

In [18]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('day',as_index=True)[['평균대기시간']].describe()

평균대기시간                                                         
     count       mean        std   min     25%    50%     75%   max
day                                                                
1     96.0  37.095833  13.795323  17.7  26.100  34.20  45.000  88.2
2     96.0  37.564583  13.423436  17.2  27.750  35.20  45.125  73.9
3     96.0  36.615625  12.768311  18.2  26.725  33.10  43.475  82.0
4     96.0  38.382292  13.888670  18.4  27.825  35.50  44.900  82.4
5     96.0  38.445833  14.610976  18.2  27.775  34.30  47.375  80.8
6     96.0  36.551042  12.771976  19.1  27.525  33.85  44.775  72.7
7     96.0  40.040625  14.211504  18.3  31.325  38.00  46.325  96.1
8     96.0  41.313542  13.955023  18.9  31.550  39.80  48.250  90.2
9     96.0  38.325000  13.063940  18.4  27.875  36.40  47.475  76.5
10    96.0  40.586458  14.250316  17.8  29.375  37.40  49.725  84.8
11    96.0  39.836458  13.516984  17.8  30.700  39.10  48.825  82.1
12    96.0  40.212500  13.568014  19.0  31.350  38.10  50.225  88.6
13    96.0  40.711458  13.856550  18.5  31.950  38.25  47.875  76.6
14    96.0  41.136458  14.930711  18.4  31.500  38.75  49.200  91.8
15    96.0  39.602083  14.572743  19.9  27.875  37.20  47.300  88.7
16    96.0  39.921875  13.154837  19.4  31.875  38.85  45.650  81.1
17    96.0  39.721875  14.056467  18.7  29.200  36.95  48.025  94.7
18    96.0  41.413542  14.690940  18.5  32.075  38.05  51.725  89.0
19    96.0  41.018750  14.777852  19.4  30.825  38.70  49.700  89.8
20    96.0  44.436458  15.898005  19.4  32.600  42.15  52.925  92.4
21    96.0  42.431250  15.791592  19.1  31.900  39.85  50.325  96.0
22    96.0  40.838542  12.774987  21.0  31.900  41.00  49.050  80.0
23    96.0  41.393750  13.962565  19.3  31.500  39.30  49.475  82.5
24    96.0  41.395833  14.398859  18.5  30.550  38.45  48.600  86.7
25    96.0  41.955208  14.749907  18.4  31.600  39.50  51.250  93.4
26    96.0  40.973958  13.970693  19.0  30.100  39.70  48.150  84.5
27    96.0  42.785417  14.709689  18.2  33.800  41.15  49.825  84.9
28    96.0  42.920833  15.024604  18.2  32.125  40.70  49.975  79.4
29    90.0  41.486667  12.939324  19.7  32.000  39.95  49.000  76.7
30    88.0  40.419318  12.220671  19.4  31.875  38.60  46.650  72.0
31    56.0  39.826786  14.314144  19.6  28.650  38.80  50.000  79.3

* 운임

In [19]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('day',as_index=True)[['평균요금']].describe()

평균요금                                                                   
    count         mean         std     min      25%     50%      75%     max
day                                                                         
1    96.0  2325.781250  107.379720  2165.0  2241.50  2277.5  2419.00  2611.0
2    96.0  2290.927083  102.115131  2159.0  2218.75  2243.0  2373.25  2525.0
3    96.0  2312.458333  106.769986  2173.0  2232.25  2256.5  2422.50  2567.0
4    96.0  2302.812500  106.519710  2166.0  2227.50  2255.5  2395.25  2588.0
5    96.0  2309.729167  114.675146  2160.0  2224.00  2257.0  2411.75  2708.0
6    96.0  2307.687500  104.567008  2168.0  2228.00  2258.5  2411.00  2528.0
7    96.0  2294.854167  101.950140  2174.0  2227.75  2248.0  2396.25  2531.0
8    96.0  2299.104167  111.892663  2151.0  2219.00  2253.5  2362.25  2587.0
9    96.0  2306.854167  108.293848  2141.0  2222.75  2259.0  2399.25  2536.0
10   96.0  2304.322917  108.803589  2177.0  2224.00  2253.5  2387.00  2578.0
11   96.0  2301.156250  107.043948  2168.0  2227.00  2255.0  2413.25  2546.0
12   96.0  2304.364583  106.796424  2157.0  2228.75  2255.0  2396.25  2526.0
13   96.0  2303.218750  107.727747  2166.0  2226.25  2263.0  2385.75  2649.0
14   96.0  2303.583333  109.555337  2172.0  2226.50  2261.5  2389.50  2616.0
15   96.0  2309.604167  115.164274  2172.0  2228.00  2259.5  2397.75  2733.0
16   96.0  2295.739583  106.726729  2131.0  2219.75  2249.5  2402.25  2625.0
17   96.0  2298.947917  102.456910  2158.0  2231.00  2251.5  2397.00  2540.0
18   96.0  2301.177083  104.586604  2146.0  2230.00  2255.5  2407.75  2542.0
19   96.0  2303.312500  110.669000  2152.0  2227.75  2258.0  2402.00  2700.0
20   96.0  2309.479167  102.031778  2165.0  2237.00  2263.0  2388.25  2559.0
21   96.0  2299.479167  104.122901  2150.0  2229.75  2255.0  2398.25  2568.0
22   96.0  2297.989583  103.505250  2169.0  2228.25  2260.5  2383.75  2565.0
23   96.0  2293.593750  107.113081  2152.0  2221.00  2246.0  2387.25  2550.0
24   96.0  2309.770833  111.518276  2144.0  2238.00  2258.5  2402.75  2628.0
25   96.0  2316.416667  116.356952  2165.0  2233.50  2263.5  2414.50  2639.0
26   96.0  2301.187500  104.203173  2161.0  2230.75  2253.5  2405.00  2528.0
27   96.0  2308.218750  110.467543  2181.0  2231.75  2256.5  2404.00  2685.0
28   96.0  2309.000000  113.397484  2161.0  2235.50  2266.0  2383.75  2690.0
29   90.0  2302.088889  103.845569  2159.0  2229.00  2263.5  2376.75  2577.0
30   88.0  2303.931818  106.667040  2168.0  2227.25  2258.0  2403.25  2565.0
31   56.0  2306.357143  104.567930  2186.0  2235.75  2265.0  2391.50  2541.0

* 이동거리

In [20]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('day',as_index=True)[['평균승차거리']].describe()

평균승차거리                                                               \
     count         mean          std     min      25%     50%       75%   
day                                                                       
1     96.0  9481.916667  1015.129711  7912.0  8681.25  9096.0  10438.50   
2     96.0  9128.427083   934.621756  7863.0  8451.00  8769.0   9959.25   
3     96.0  9331.250000  1016.939091  8074.0  8572.25  8857.5  10330.75   
4     96.0  9242.875000  1011.352104  7942.0  8502.50  8799.0  10000.50   
5     96.0  9310.510417  1089.267054  7808.0  8506.50  8825.5  10244.00   
6     96.0  9294.281250  1001.189899  7979.0  8537.25  8849.0  10283.00   
7     96.0  9169.781250   994.621443  7972.0  8476.50  8748.5  10092.50   
8     96.0  9199.520833  1091.878295  7824.0  8462.00  8759.0   9714.75   
9     96.0  9295.677083  1054.210041  7695.0  8504.00  8822.5  10307.50   
10    96.0  9253.125000  1042.698687  8049.0  8490.25  8766.0  10199.00   
11    96.0  9215.364583  1002.938064  7995.0  8492.75  8806.0  10171.25   
12    96.0  9242.437500   994.127936  7806.0  8601.75  8783.5  10105.25   
13    96.0  9233.635417  1009.707577  7936.0  8468.75  8814.0  10021.00   
14    96.0  9246.458333  1050.308225  7967.0  8544.50  8802.5  10276.50   
15    96.0  9290.833333  1075.825164  7888.0  8525.75  8861.5  10142.75   
16    96.0  9170.187500   979.771760  7672.0  8452.50  8747.0  10134.25   
17    96.0  9200.656250   968.370077  7879.0  8534.00  8844.5   9973.50   
18    96.0  9220.250000  1018.676355  7748.0  8533.00  8820.5  10117.50   
19    96.0  9244.833333  1095.999353  7872.0  8543.00  8850.0  10014.50   
20    96.0  9313.395833   984.920659  7957.0  8590.75  8933.0  10242.00   
21    96.0  9202.822917   990.256160  7832.0  8509.50  8767.0   9867.50   
22    96.0  9184.197917   977.007565  7916.0  8517.00  8843.5   9838.25   
23    96.0  9149.520833   983.159739  7850.0  8487.00  8754.5  10112.25   
24    96.0  9296.562500  1035.514089  7797.0  8581.00  8865.5  10155.00   
25    96.0  9387.770833  1095.206238  7939.0  8626.75  8920.5  10243.50   
26    96.0  9239.156250  1001.731851  7889.0  8524.75  8832.5  10173.50   
27    96.0  9283.250000  1071.473074  8049.0  8538.25  8850.0  10179.25   
28    96.0  9299.958333  1097.055618  7882.0  8565.50  8855.5  10138.00   
29    90.0  9211.133333   962.155331  7847.0  8492.50  8826.5   9670.25   
30    88.0  9248.295455   997.315986  7910.0  8491.00  8820.0  10185.25   
31    56.0  9277.214286   990.723312  8094.0  8542.00  8895.0  10251.25   

              
         max  
day           
1    12064.0  
2    11092.0  
3    11747.0  
4    11837.0  
5    12913.0  
6    11479.0  
7    11526.0  
8    13056.0  
9    12265.0  
10   11856.0  
11   11571.0  
12   11701.0  
13   12516.0  
14   12213.0  
15   13304.0  
16   12257.0  
17   11404.0  
18   11992.0  
19   14136.0  
20   11864.0  
21   11822.0  
22   11423.0  
23   11445.0  
24   12145.0  
25   12404.0  
26   11700.0  
27   13904.0  
28   12747.0  
29   11586.0  
30   11540.0  
31   11316.0

### (3) 요일별

* 차량 운행수

In [21]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('weekday',as_index=True)[['차량운행']].describe()

차량운행                                                             
           count        mean        std    min     25%    50%     75%     max
weekday                                                                      
Friday     418.0  485.107656  82.011846  178.0  444.00  488.5  539.00   690.0
Monday     417.0  485.014388  87.866840  197.0  443.00  485.0  543.00   678.0
Saturday   418.0  262.177033  35.577150  161.0  233.25  260.0  291.75   345.0
Sunday     417.0  224.767386  34.109708  132.0  201.00  218.0  245.00   333.0
Thursday   418.0  494.078947  89.615625  182.0  446.00  493.5  539.75  1413.0
Tuesday    417.0  495.076739  80.615743  182.0  449.00  500.0  541.00   814.0
Wednesday  417.0  484.726619  84.216908  197.0  440.00  486.0  540.00   692.0

* 접수건, 탑승건

In [22]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('weekday',as_index=True)[['접수건','탑승건']].describe()

접수건                                                            \
           count         mean         std     min      25%     50%     75%   
weekday                                                                      
Friday     418.0  4692.066986  894.851008   805.0  4533.50  4940.5  5171.0   
Monday     417.0  4715.040767  931.581903  1127.0  4597.00  4986.0  5208.0   
Saturday   418.0  2069.081340  326.397821   856.0  1908.00  2157.5  2298.0   
Sunday     417.0  1702.781775  376.066110   527.0  1500.00  1869.0  1952.0   
Thursday   418.0  4760.677033  841.902424  1023.0  4614.75  4964.0  5178.0   
Tuesday    417.0  4800.884892  870.787606  1287.0  4706.00  5026.0  5233.0   
Wednesday  417.0  4739.333333  923.886493  1327.0  4613.00  4996.0  5226.0   

                     탑승건                                                   \
              max  count         mean         std     min     25%     50%   
weekday                                                                     
Friday     5990.0  418.0  3877.385167  727.064752   729.0  3778.5  4041.5   
Monday     6100.0  417.0  3961.242206  787.612615  1004.0  3908.0  4161.0   
Saturday   2794.0  418.0  1693.839713  227.514641   770.0  1583.0  1723.5   
Sunday     2154.0  417.0  1493.556355  303.130807   462.0  1367.0  1610.0   
Thursday   6133.0  418.0  3978.200957  694.129880   924.0  3868.0  4096.0   
Tuesday    6134.0  417.0  4026.791367  730.969389  1046.0  3939.0  4180.0   
Wednesday  6182.0  417.0  3956.976019  765.748662  1156.0  3872.0  4119.0   

                            
               75%     max  
weekday                     
Friday     4244.75  5025.0  
Monday     4372.00  5151.0  
Saturday   1848.75  2199.0  
Sunday     1695.00  1889.0  
Thursday   4332.75  5129.0  
Tuesday    4412.00  5130.0  
Wednesday  4362.00  5189.0

* 대기시간

In [23]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('weekday',as_index=True)[['평균대기시간']].describe()

평균대기시간                                                        
           count       mean        std   min     25%   50%     75%   max
weekday                                                                 
Friday     418.0  40.956459  13.380374  17.2  32.175  39.3  48.950  96.1
Monday     417.0  39.243165  12.598604  18.4  29.500  37.6  46.600  82.0
Saturday   418.0  43.480144  16.476962  17.6  29.525  41.5  53.750  94.7
Sunday     417.0  34.902878  12.619141  17.8  24.200  32.9  41.700  80.8
Thursday   418.0  41.043780  13.444326  17.8  31.300  39.6  48.525  92.4
Tuesday    417.0  40.826139  13.776295  18.2  30.600  38.6  48.900  89.0
Wednesday  417.0  41.676259  14.542795  18.8  31.000  38.8  50.900  89.8

* 운임

In [24]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('weekday',as_index=True)[['평균요금']].describe()

평균요금                                                           \
           count         mean        std     min     25%     50%      75%   
weekday                                                                     
Friday     418.0  2246.023923  55.505433  2131.0  2216.0  2240.0  2261.00   
Monday     417.0  2240.059952  55.938271  2146.0  2212.0  2233.0  2254.00   
Saturday   418.0  2429.641148  65.626115  2255.0  2383.5  2431.0  2473.75   
Sunday     417.0  2464.541966  46.598415  2284.0  2435.0  2463.0  2491.00   
Thursday   418.0  2256.366029  60.688792  2169.0  2228.0  2251.0  2268.75   
Tuesday    417.0  2242.805755  59.983116  2141.0  2213.0  2238.0  2258.00   
Wednesday  417.0  2250.422062  51.987372  2150.0  2223.0  2244.0  2264.00   

                   
              max  
weekday            
Friday     2649.0  
Monday     2628.0  
Saturday   2690.0  
Sunday     2685.0  
Thursday   2733.0  
Tuesday    2708.0  
Wednesday  2588.0

* 이동거리

In [25]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('weekday',as_index=True)[['평균승차거리']].describe()

평균승차거리                                                       \
           count          mean         std     min       25%      50%   
weekday                                                                 
Friday     418.0   8733.665072  558.522810  7672.0   8441.50   8661.5   
Monday     417.0   8656.577938  589.888149  7748.0   8335.00   8583.0   
Saturday   418.0  10431.468900  638.927763  8714.0  10013.75  10442.0   
Sunday     417.0  10741.275779  447.586297  8987.0  10467.00  10722.0   
Thursday   418.0   8786.901914  601.413752  7916.0   8495.50   8727.5   
Tuesday    417.0   8672.112710  584.911046  7695.0   8373.00   8618.0   
Wednesday  417.0   8752.038369  517.201469  7832.0   8460.00   8683.0   

                              
                75%      max  
weekday                       
Friday      8901.25  12516.0  
Monday      8820.00  13056.0  
Saturday   10892.00  12747.0  
Sunday     11059.00  13904.0  
Thursday    8953.75  14136.0  
Tuesday     8815.00  12913.0  
Wednesday   8933.00  11992.0

### (4) 월별

* 차량 운행수

In [26]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('month',as_index=True)[['차량운행']].describe()

차량운행                                                              
       count        mean         std    min     25%    50%     75%     max
month                                                                     
1      248.0  379.286290  125.021855  132.0  245.00  411.0  444.25   814.0
2      226.0  373.486726  125.424549  167.0  235.50  409.5  446.00   595.0
3      248.0  398.758065  123.967958  161.0  263.25  432.5  474.00   615.0
4      240.0  416.187500  123.552685  188.0  271.75  461.0  497.00   601.0
5      248.0  413.556452  128.678289  171.0  273.75  453.0  516.75   608.0
6      240.0  430.925000  139.803009  196.0  285.25  469.5  519.00  1413.0
7      248.0  436.935484  125.948723  182.0  292.25  477.5  526.25   619.0
8      248.0  434.500000  130.694842  189.0  282.00  484.0  527.50   685.0
9      240.0  446.412500  142.029671  201.0  295.00  498.5  542.00   700.0
10     248.0  432.939516  135.852595  173.0  289.00  491.0  531.00   661.0
11     240.0  450.483333  137.449206  176.0  302.50  493.0  547.00   684.0
12     248.0  409.173387  142.082167  139.0  264.75  426.0  493.00   697.0

* 접수건, 탑승건

In [27]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('month',as_index=True)[['접수건','탑승건']].describe()

접수건                                                              \
       count         mean          std     min      25%     50%      75%   
month                                                                      
1      248.0  3681.826613  1491.384086   591.0  1918.25  4407.5  4949.50   
2      226.0  3582.097345  1485.482395   856.0  1898.50  4411.0  4850.50   
3      248.0  3691.330645  1508.082608   527.0  2077.25  4611.0  4881.00   
4      240.0  3964.479167  1485.197345   623.0  2209.50  4867.5  5087.25   
5      248.0  3910.572581  1509.734828  1106.0  2171.00  4839.5  5117.50   
6      240.0  4001.304167  1475.040740  1099.0  2309.50  4850.5  5169.00   
7      248.0  4099.637097  1443.957079   859.0  2322.00  4724.0  5115.25   
8      248.0  4014.963710  1494.198219   619.0  2273.75  4778.5  5172.00   
9      240.0  3962.508333  1598.211540   655.0  2111.00  4864.0  5250.25   
10     248.0  4007.709677  1589.318578   967.0  2153.25  4933.5  5240.75   
11     240.0  4243.991667  1448.449533   951.0  2336.50  4955.0  5222.75   
12     248.0  3931.556452  1485.666749   609.0  2183.25  4714.5  5011.25   

                 탑승건                                                     \
          max  count         mean          std     min      25%     50%   
month                                                                     
1      5493.0  248.0  3092.262097  1215.867502   551.0  1640.00  3753.5   
2      5240.0  226.0  3032.075221  1244.758189   770.0  1629.25  3744.5   
3      5407.0  248.0  3168.822581  1285.173427   462.0  1742.00  3978.0   
4      6075.0  240.0  3358.558333  1238.475408   591.0  1827.75  4085.5   
5      5861.0  248.0  3278.435484  1271.347637  1012.0  1783.25  4005.0   
6      6100.0  240.0  3372.837500  1229.861295  1013.0  1855.00  3939.0   
7      6092.0  248.0  3437.862903  1184.486245   809.0  1904.75  3981.0   
8      6134.0  248.0  3350.512097  1227.490920   580.0  1841.00  4002.5   
9      6182.0  240.0  3287.950000  1302.137389   618.0  1783.25  4004.5   
10     6164.0  248.0  3316.850806  1307.951072   908.0  1776.50  4009.5   
11     5959.0  240.0  3498.500000  1213.246359   891.0  1827.75  4042.5   
12     6128.0  248.0  3202.068548  1208.429194   580.0  1699.25  3713.0   

                        
           75%     max  
month                   
1      4013.50  4504.0  
2      4030.25  4496.0  
3      4201.50  4665.0  
4      4348.25  4882.0  
5      4352.75  4850.0  
6      4394.75  4983.0  
7      4248.50  5083.0  
8      4239.75  5120.0  
9      4271.50  5189.0  
10     4345.00  5106.0  
11     4337.50  5000.0  
12     4114.25  4999.0

* 대기시간

In [28]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('month',as_index=True)[['평균대기시간']].describe()

평균대기시간                                                         
       count       mean        std   min     25%    50%     75%   max
month                                                                
1      248.0  33.393952  10.882170  17.2  25.975  31.60  39.025  76.7
2      226.0  34.419469  13.158108  18.3  24.725  29.45  40.275  78.9
3      248.0  35.182258  14.182723  18.2  24.075  30.40  44.650  74.1
4      240.0  39.628750  13.241641  20.1  29.000  36.60  47.900  79.9
5      248.0  41.181452  12.241648  19.7  32.300  39.45  48.725  80.8
6      240.0  40.381667  12.817682  17.8  31.600  38.75  48.675  77.0
7      248.0  40.033468  11.055332  18.4  32.225  38.75  46.400  69.7
8      248.0  38.568952  11.605539  18.2  30.675  37.75  45.125  84.5
9      240.0  42.364167  15.125236  18.2  30.350  42.20  52.675  89.2
10     248.0  44.480242  13.215172  18.5  36.800  42.60  52.400  83.7
11     240.0  47.063333  14.913303  20.1  36.100  43.95  55.425  94.7
12     248.0  46.713306  17.583373  18.5  35.850  42.75  55.950  96.1

* 운임

In [29]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('month',as_index=True)[['평균요금']].describe()

평균요금                                                             \
       count         mean         std     min      25%     50%      75%   
month                                                                     
1      248.0  2279.616935  106.003793  2146.0  2204.75  2230.0  2372.00   
2      226.0  2296.623894  109.799717  2158.0  2217.00  2246.0  2382.00   
3      248.0  2291.600806   97.659648  2172.0  2224.00  2244.0  2360.75   
4      240.0  2310.712500   99.238000  2157.0  2238.75  2261.0  2399.75   
5      248.0  2326.209677  102.864029  2164.0  2252.00  2277.5  2433.00   
6      240.0  2308.991667  103.091339  2173.0  2239.00  2261.0  2416.00   
7      248.0  2290.407258   97.705247  2152.0  2226.00  2251.0  2380.00   
8      248.0  2286.189516   98.496143  2141.0  2217.75  2246.0  2366.50   
9      240.0  2331.583333  123.131259  2160.0  2242.00  2270.0  2440.75   
10     248.0  2334.774194  113.621777  2180.0  2250.50  2275.0  2446.25   
11     240.0  2309.175000  110.397165  2168.0  2235.00  2257.5  2406.50   
12     248.0  2286.104839  106.666290  2131.0  2212.75  2235.0  2379.00   

               
          max  
month          
1      2690.0  
2      2708.0  
3      2526.0  
4      2544.0  
5      2537.0  
6      2565.0  
7      2551.0  
8      2573.0  
9      2733.0  
10     2640.0  
11     2578.0  
12     2554.0

* 이동거리

In [30]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('month',as_index=True)[['평균승차거리']].describe()

평균승차거리                                                               \
       count         mean          std     min      25%     50%       75%   
month                                                                       
1      248.0  9048.161290  1016.645061  7748.0  8311.25  8634.0  10038.25   
2      226.0  9221.039823  1113.085047  7863.0  8413.00  8744.5  10009.00   
3      248.0  9129.588710   930.761627  7967.0  8453.00  8742.5   9710.25   
4      240.0  9313.458333   957.384844  7806.0  8580.00  8970.0  10099.00   
5      248.0  9462.447581  1000.402602  7903.0  8728.50  9042.5  10338.00   
6      240.0  9292.650000   961.064143  7993.0  8622.50  8875.5  10265.75   
7      248.0  9151.556452   934.211901  7872.0  8513.75  8759.0   9993.50   
8      248.0  9127.072581   954.195646  7695.0  8456.00  8756.5   9813.25   
9      240.0  9493.016667  1174.640641  7808.0  8630.00  8993.5  10463.75   
10     248.0  9485.576613  1048.753535  8027.0  8695.75  8960.5  10503.75   
11     240.0  9251.379167  1023.729784  7910.0  8567.50  8789.5  10314.75   
12     248.0  9074.028226   982.823205  7672.0  8385.00  8662.0   9917.00   

                
           max  
month           
1      12747.0  
2      14136.0  
3      11490.0  
4      11701.0  
5      12230.0  
6      11421.0  
7      11428.0  
8      11642.0  
9      13904.0  
10     12228.0  
11     11680.0  
12     11563.0

### (5) 연도별

* 차량 운행수

In [31]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('year',as_index=True)[['차량운행']].describe()

차량운행                                                            
      count        mean         std    min    25%    50%    75%     max
year                                                                   
2015  365.0  381.468493  107.000499  163.0  252.0  440.0  460.0   507.0
2016  366.0  390.314208  128.274949  164.0  252.5  440.0  482.0  1413.0
2017  365.0  405.397260  125.298341  132.0  260.0  464.0  509.0   548.0
2018  365.0  397.758904  123.445819  164.0  270.0  434.0  507.0   570.0
2019  365.0  399.068493  125.626335  161.0  269.0  449.0  504.0   814.0
2020  366.0  416.521858  124.615104  185.0  296.0  435.0  516.5   654.0
2021  365.0  468.271233  133.933703  190.0  306.0  543.0  567.0   615.0
2022  365.0  490.898630  156.483781  209.0  304.0  558.0  604.0   700.0

* 접수건, 탑승건

In [32]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_taxi.groupby('year',as_index=True)[['접수건','탑승건']].describe()

접수건                                                              \
      count         mean          std     min      25%     50%      75%   
year                                                                      
2015  365.0  3750.531507  1286.489660  1023.0  2149.00  4471.0  4757.00   
2016  366.0  4005.087432  1399.214667  1319.0  2229.25  4837.5  5054.25   
2017  365.0  4184.745205  1489.248040  1478.0  2259.00  5123.0  5297.00   
2018  365.0  4077.473973  1433.818602  1419.0  2181.00  4985.0  5120.00   
2019  365.0  4104.358904  1427.305709  1406.0  2246.00  4965.0  5141.00   
2020  366.0  3137.480874  1387.445751   527.0  1729.00  3593.5  4318.75   
2021  365.0  3796.821918  1584.925666   591.0  1978.00  4666.0  5077.00   
2022  365.0  4350.304110  1714.018398  1057.0  2362.00  5117.0  5764.00   

                탑승건                                                     \
         max  count         mean          std     min      25%     50%   
year                                                                     
2015  5379.0  365.0  3268.673973  1123.983935   924.0  1803.00  3938.0   
2016  5769.0  366.0  3418.183060  1193.308077  1149.0  1832.25  4122.0   
2017  5955.0  365.0  3471.709589  1246.805403  1203.0  1849.00  4240.0   
2018  5605.0  365.0  3209.380822  1129.004514  1158.0  1700.00  3907.0   
2019  5614.0  365.0  3230.279452  1124.524067  1046.0  1728.00  3890.0   
2020  5333.0  366.0  2656.314208  1122.990558   462.0  1493.25  3149.0   
2021  5572.0  365.0  3344.109589  1361.665167   551.0  1720.00  4123.0   
2022  6182.0  365.0  3673.863014  1415.591273   974.0  1933.00  4397.0   

                      
         75%     max  
year                  
2015  4122.0  4450.0  
2016  4326.0  4700.0  
2017  4447.0  4882.0  
2018  4079.0  4325.0  
2019  4081.0  4363.0  
2020  3610.0  4166.0  
2021  4408.0  4816.0  
2022  4797.0  5189.0

## 3.데이터 구조 만들기

* **세부요구사항**
    * 조건 : 
        * 목표 : 전날 저녁, 다음날 평균 대기시간을 예측하고자 합니다.
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
    * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙입니다.

In [33]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi.rename(columns={'기준일':'Date'},inplace=True)
taxi.head()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


### (1) target 만들기
* 예측하는 날짜, 대기시간(target)으로 기준을 잡습니다.

In [34]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 대기시간(waiting time)을 오늘의 데이터를 활용하여 예측 해야하는 대상(target)으로 설정
taxi['target']=taxi['평균대기시간'].shift(-1)
taxi.tail()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target
2917,2022-12-27,669,5635,4654,44.4,2198,8178,44.8
2918,2022-12-28,607,5654,4648,44.8,2161,7882,52.5
2919,2022-12-29,581,5250,4247,52.5,2229,8433,38.3
2920,2022-12-30,600,5293,4200,38.3,2183,8155,33.7
2921,2022-12-31,263,2167,1806,33.7,2318,9435,NaN


### (2) 날씨 데이터 붙이기
* merge를 활용합니다. 기준은 운행정보 입니다.

In [35]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 실제 날씨 데이터를 전일에 발표된 예보데이터로 판단
weather['Date']+=pd.Timedelta(days=-1)
data=pd.merge(taxi,weather)

In [36]:
weather.tail()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
4013,2022-12-26,3.0,-7.3,0.0,86.0,51.0,10.25
4014,2022-12-27,-0.3,-5.4,0.1,92.0,40.0,10.86
4015,2022-12-28,1.7,-7.8,0.0,71.0,34.0,10.88
4016,2022-12-29,2.1,-4.0,0.0,87.0,38.0,10.84
4017,2022-12-30,-4.4,-4.4,0.0,66.0,66.0,0.00


In [37]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data.head()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,0.0,63.0,28.0,9.07
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,0.0,73.0,37.0,8.66
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,0.0,89.0,58.0,5.32
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,0.0,95.0,52.0,6.48
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,3.4,98.0,29.0,10.47


### (3) 새로운 feature를 생성해 봅시다.
* 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 그외 새로운 feature 도출 : 최소 2개 이상
    * 예 : 공휴일, 최근 7주일간의 평균 대기시간, 탑승률 등

#### 1) 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 요일 이름, 계절이름, 월 이름으로 만드는 경우에는, 변수를 pd.Categorical로 범주형을 만들면서 순서를 지정하는 것이 이후 그래프를 그릴 때 순서대로 표현할 수 있습니다.


In [38]:
data['weekday'] = data['Date'].dt.day_name()
data['weekday'] = pd.Categorical(data['weekday'], 
                                  categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
data['month'] = data['Date'].dt.month

data['season'] = np.where(data['month'].isin([3,4,5]), 'Spring',
                           np.where(data['month'].isin([6,7,8]), 'Summer',
                                    np.where(data['month'].isin([9,10,11]), 'Fall', 'Winter')))
data['season'] = pd.Categorical(data['season'], categories=['Spring','Summer','Fall','Winter'])
                                     
data['year'] = data['Date'].dt.year
data

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,0.0,63.0,28.0,9.07,Thursday,1,Winter,2015
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,0.0,73.0,37.0,8.66,Friday,1,Winter,2015
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,0.0,89.0,58.0,5.32,Saturday,1,Winter,2015
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,0.0,95.0,52.0,6.48,Sunday,1,Winter,2015
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,3.4,98.0,29.0,10.47,Monday,1,Winter,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,2022-12-26,603,5555,4605,39.2,2163,7889,44.4,3.0,-7.3,0.0,86.0,51.0,10.25,Monday,12,Winter,2022
2917,2022-12-27,669,5635,4654,44.4,2198,8178,44.8,-0.3,-5.4,0.1,92.0,40.0,10.86,Tuesday,12,Winter,2022
2918,2022-12-28,607,5654,4648,44.8,2161,7882,52.5,1.7,-7.8,0.0,71.0,34.0,10.88,Wednesday,12,Winter,2022
2919,2022-12-29,581,5250,4247,52.5,2229,8433,38.3,2.1,-4.0,0.0,87.0,38.0,10.84,Thursday,12,Winter,2022


#### 2) 공휴일 정보
* workalendar 패키지를 설치하고, 대한민국 공휴일 정보를 끌어와 봅시다.

* 휴무일 데이터 패키지 설치

In [39]:
!pip install workalendar

* 간단 사용법

In [40]:
from workalendar.asia import SouthKorea
cal = SouthKorea()
pd.DataFrame(cal.holidays(2023))

,0,1
0,2023-01-01,New year
1,2023-01-21,Korean New Year's Day
2,2023-01-22,Korean New Year's Day
3,2023-01-23,Korean New Year's Day
4,2023-03-01,Independence Day
5,2023-05-05,Children's Day
6,2023-05-26,Buddha's Birthday
7,2023-06-06,Memorial Day
8,2023-08-15,Liberation Day
9,2023-09-28,Midautumn Festival


* 휴무일 데이터셋 만들기 2015 ~ 2022
* 실제로 휴무일에 해당하지만 workalendar 라이브러리에 없는 날짜는 직접 추가해봅시다.
    * 휴무일 장애인 콜택시의 접수건 변화에 대한 특징을 찾아 이를 바탕으로 데이터를 조회하여 찾아볼 수 있음

In [41]:
from workalendar.asia import SouthKorea

cal = SouthKorea()
holiday = pd.DataFrame()
for y in range(2015, 2023) :
    holiday = pd.concat([holiday, pd.DataFrame(cal.holidays(y))], axis = 0)

holiday.columns = ['Date', 'holiday']
holiday['Date'] = pd.to_datetime(holiday['Date'])
holiday['holiday'] = 1
holiday

,Date,holiday
0,2015-01-01,1
1,2015-02-18,1
2,2015-02-19,1
3,2015-02-20,1
4,2015-03-01,1
...,...,...
10,2022-09-10,1
11,2022-09-11,1
12,2022-10-03,1
13,2022-10-09,1


* 기존 데이터에 휴무일 정보 결합하기.
* 휴무일이 아닌 날짜는 0으로 저장하시오.

In [42]:
data = pd.merge(data, holiday, how = 'left')
data = data.fillna({'holiday':0})

In [43]:
data.tail()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year,holiday
2917,2022-12-26,603,5555,4605,39.2,2163,7889,44.4,3.0,-7.3,0.0,86.0,51.0,10.25,Monday,12,Winter,2022,0.0
2918,2022-12-27,669,5635,4654,44.4,2198,8178,44.8,-0.3,-5.4,0.1,92.0,40.0,10.86,Tuesday,12,Winter,2022,0.0
2919,2022-12-28,607,5654,4648,44.8,2161,7882,52.5,1.7,-7.8,0.0,71.0,34.0,10.88,Wednesday,12,Winter,2022,0.0
2920,2022-12-29,581,5250,4247,52.5,2229,8433,38.3,2.1,-4.0,0.0,87.0,38.0,10.84,Thursday,12,Winter,2022,0.0
2921,2022-12-30,600,5293,4200,38.3,2183,8155,33.7,-4.4,-4.4,0.0,66.0,66.0,0.00,Friday,12,Winter,2022,0.0


#### 3) 7일 이동평균 대기시간
* rolling().mean() 사용

In [44]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data['7일간 평균대기시간']=data[['평균대기시간']].rolling(window=7).mean()

In [53]:
data.head()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target,temp_max,temp_min,...,sunshine(MJ/m2),weekday,month,season,year,holiday,7일간 평균대기시간,탑승률,차량당 접수건수,차량당 탑승건수
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,...,9.07,Thursday,1,Winter,2015,1.0,NaN,0.903226,4.802817,4.338028
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,...,8.66,Friday,1,Winter,2015,0.0,NaN,0.898987,7.519048,6.759524
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,...,5.32,Saturday,1,Winter,2015,0.0,NaN,0.918689,7.885167,7.244019
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,...,6.48,Sunday,1,Winter,2015,0.0,NaN,0.927096,8.397959,7.785714
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,...,10.47,Monday,1,Winter,2015,0.0,NaN,0.877647,10.095012,8.859857


#### 4) 탑승률

In [46]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data['탑승률']=data['탑승건']/data['접수건']

In [47]:
data.head()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target,temp_max,temp_min,...,humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year,holiday,7일간 평균대기시간,탑승률
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,...,63.0,28.0,9.07,Thursday,1,Winter,2015,1.0,NaN,0.903226
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,...,73.0,37.0,8.66,Friday,1,Winter,2015,0.0,NaN,0.898987
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,...,89.0,58.0,5.32,Saturday,1,Winter,2015,0.0,NaN,0.918689
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,...,95.0,52.0,6.48,Sunday,1,Winter,2015,0.0,NaN,0.927096
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,...,98.0,29.0,10.47,Monday,1,Winter,2015,0.0,NaN,0.877647


#### 5) 차량운행당 평균대기시간

In [48]:
data['차량당 접수건수']=data['접수건']/data['차량운행']
data['차량당 탑승건수']=data['탑승건']/data['차량운행']

In [49]:
data.head()

,Date,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,target,temp_max,temp_min,...,sunshine(MJ/m2),weekday,month,season,year,holiday,7일간 평균대기시간,탑승률,차량당 접수건수,차량당 탑승건수
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,...,9.07,Thursday,1,Winter,2015,1.0,NaN,0.903226,4.802817,4.338028
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,...,8.66,Friday,1,Winter,2015,0.0,NaN,0.898987,7.519048,6.759524
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,...,5.32,Saturday,1,Winter,2015,0.0,NaN,0.918689,7.885167,7.244019
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,...,6.48,Sunday,1,Winter,2015,0.0,NaN,0.927096,8.397959,7.785714
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,...,10.47,Monday,1,Winter,2015,0.0,NaN,0.877647,10.095012,8.859857


## 4.데이터 저장
* **세부요구사항**
    * joblib 을 사용하여 작업 경로에 정리한 데이터프레임을 저장합니다.
        * 저장파일이름 : data1.pkl

In [50]:
data=data.drop(data.index[-1])

In [51]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
joblib.dump(data,'data1.pkl')

['data1.pkl']

In [52]:
data.to_csv('data1.csv')

## 2. 데이터 분석

## 3. 모델링 및 가설 검증